# About
**Goal:** Demonstrate extraction, transformation, and load use-cases for nonprofit/foundations.

**Actions:**

- Collect social media data on two foundations.
- Transform data into a human readable format 
- Visualize a trend from the data.

**Audience:** Intermediate level Tensorflow users.

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import json
import time
import math
import numpy as np

from tweepy import Cursor
from twitter_client import get_twitter_client
from dotenv import load_dotenv, find_dotenv
from etl import usage, paginate, extract, generate_friends_df, generate_followers_df

from highcharts import Highchart
from visualization import options

# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()
# load up the entries as environment variables
load_dotenv(dotenv_path)

True

In [2]:
foundations = ['opensociety', 'RobinHoodNYC']

In [3]:
# extract(foundations[0])

## Robinhood Foundation (NYC)

### [Fighting Poverty in New York](https://www.robinhood.org/)

**Mission:** 
Robin Hood’s mission is to improve the living standards for 1.8 million low-income new yorkers.

**Objective:** Robin Hood has a single objective: end poverty in NYC.

#### Friends

In [4]:
friends_df = generate_friends_df('RobinHoodNYC')
friends_df.head()

,created_at,description,favourites_count,followers_count,friends_count,id,listed_count,location,name,profile_image_url,relationship,screen_name,source,statuses_count,url
0,Wed Apr 20 10:17:07 +0000 2011,@GMA News Anchor. My book #Better is available...,1150,123943,392,284991184,673,NA,Amy Robach,http://pbs.twimg.com/profile_images/5828605865...,friend,arobach,RobinHoodNYC,2416,http://t.co/9bNKrbXh3e
1,Thu Jul 23 03:41:58 +0000 2009,"Ex-@MikeBloomberg @SenSchumer chief spokesman,...",1118,7949,5970,59351766,190,Offices in NYC & CT,Stu Loeser,http://pbs.twimg.com/profile_images/6823178121...,friend,stuloeser,RobinHoodNYC,12369,https://t.co/6sarNF5kyM
2,Fri Mar 18 18:36:02 +0000 2011,#Malibu - out now! https://t.co/7jlWgcZaoE,81,32325243,373,268414482,58788,@happyhippiefdn,Miley Ray Cyrus,http://pbs.twimg.com/profile_images/8626999090...,friend,MileyCyrus,RobinHoodNYC,8499,https://t.co/w5oy1cN4nj
3,Tue Mar 03 01:17:55 +0000 2009,https://t.co/Bf9BwnlC3d 1/3 of Ocean Park Stan...,8557,1490012,1203,22563769,6901,Los Angeles,samantha ronson,http://pbs.twimg.com/profile_images/8395457478...,friend,samantharonson,RobinHoodNYC,32783,https://t.co/G8RfqOi5RL
4,Thu Jul 14 21:57:32 +0000 2011,Small business reporter at The Associated Pres...,87,2624,69,335567840,181,NA,Joyce M. Rosenberg,http://pbs.twimg.com/profile_images/1877781210...,friend,JoyceMRosenberg,RobinHoodNYC,751,None


#### Followers

In [5]:
robinhood_followers = generate_friends_df('RobinHoodNYC')
robinhood_followers.head()

,created_at,description,favourites_count,followers_count,friends_count,id,listed_count,location,name,profile_image_url,relationship,screen_name,source,statuses_count,url
0,Wed Apr 20 10:17:07 +0000 2011,@GMA News Anchor. My book #Better is available...,1150,123943,392,284991184,673,NA,Amy Robach,http://pbs.twimg.com/profile_images/5828605865...,friend,arobach,RobinHoodNYC,2416,http://t.co/9bNKrbXh3e
1,Thu Jul 23 03:41:58 +0000 2009,"Ex-@MikeBloomberg @SenSchumer chief spokesman,...",1118,7949,5970,59351766,190,Offices in NYC & CT,Stu Loeser,http://pbs.twimg.com/profile_images/6823178121...,friend,stuloeser,RobinHoodNYC,12369,https://t.co/6sarNF5kyM
2,Fri Mar 18 18:36:02 +0000 2011,#Malibu - out now! https://t.co/7jlWgcZaoE,81,32325243,373,268414482,58788,@happyhippiefdn,Miley Ray Cyrus,http://pbs.twimg.com/profile_images/8626999090...,friend,MileyCyrus,RobinHoodNYC,8499,https://t.co/w5oy1cN4nj
3,Tue Mar 03 01:17:55 +0000 2009,https://t.co/Bf9BwnlC3d 1/3 of Ocean Park Stan...,8557,1490012,1203,22563769,6901,Los Angeles,samantha ronson,http://pbs.twimg.com/profile_images/8395457478...,friend,samantharonson,RobinHoodNYC,32783,https://t.co/G8RfqOi5RL
4,Thu Jul 14 21:57:32 +0000 2011,Small business reporter at The Associated Pres...,87,2624,69,335567840,181,NA,Joyce M. Rosenberg,http://pbs.twimg.com/profile_images/1877781210...,friend,JoyceMRosenberg,RobinHoodNYC,751,None


## Open Society Foundations

### [The Open Society Foundations work to build vibrant and tolerant democracies whose governments are accountable to their citizens](https://www.opensocietyfoundations.org/)

**Mission:** 

The Open Society Foundations work to build vibrant and tolerant societies whose governments are accountable and open to the participation of all people.

- We seek to strengthen the rule of law; respect for human rights, minorities, and a diversity of opinions; democratically elected governments; and a civil society that helps keep government power in check.

- We help to shape public policies that assure greater fairness in political, legal, and economic systems and safeguard fundamental rights.

- We implement initiatives to advance justice, education, public health, and independent media.

- We build alliances across borders and continents on issues such as corruption and freedom of information.

- Working in every part of the world, the Open Society Foundations place a high priority on protecting and improving the lives of people in marginalized communities.

In [6]:
open_society_followers = generate_friends_df('opensociety')
open_society_followers.head()

,created_at,description,favourites_count,followers_count,friends_count,id,listed_count,location,name,profile_image_url,relationship,screen_name,source,statuses_count,url
0,Mon Apr 04 13:24:51 +0000 2011,Comms @OpenSociety. Former Ukraine corresponde...,2744,12667,3476,276979950,807,"London, England",Maxim Tucker,http://pbs.twimg.com/profile_images/6764622554...,friend,MaxRTucker,opensociety,6157,https://t.co/yAVMwJqM6E
1,Wed Mar 29 18:31:54 +0000 2017,The Museum of Drug Policy is a pop-up arts & c...,16,63,80,847154362761842688,1,"Montréal, Québec (May 15-17)",#DrugPolicyMuseum,http://pbs.twimg.com/profile_images/8515411374...,friend,MofDP,opensociety,31,https://t.co/yBlVCZwRWn
2,Wed Mar 24 18:33:51 +0000 2010,America's premier legal organization fighting ...,9407,41491,3280,126062842,900,NYC & DC,Legal Defense Fund,http://pbs.twimg.com/profile_images/4895109955...,friend,NAACP_LDF,opensociety,19548,http://t.co/XAJAQLaTrd
3,Wed Apr 19 01:47:34 +0000 2017,Communities Against Hate is a diverse coalitio...,13,40,21,854511761730539521,0,United States,#AgainstHate,http://pbs.twimg.com/profile_images/8605981853...,friend,CAHTweets,opensociety,18,https://t.co/3Fth9Ic78i
4,Thu Feb 26 03:42:28 +0000 2009,NA,33,1056,713,21965684,32,NA,Michael Waldman,http://pbs.twimg.com/profile_images/3788000001...,friend,mawaldman,opensociety,163,None


# Visualize

In [7]:
def prep_data(x, y, data, name):
    d = data[[x, y]].fillna(1)
    frame = d.values.tolist()
    return {'name': name, 'data': frame}

def plot_scatter(group1, group2):
    H = Highchart(width=850, height=400)
    H.set_dict_options(options)
    H.add_data_set(group1['data'], 'scatter', group1['name'], color='rgba(223, 83, 83, .5)')
    H.add_data_set(group2['data'], 'scatter', group2['name'], color='rgba(119, 152, 191, .5)')
    return H

# Compare Robin Hood Foundation to Open Society Foundation

In [8]:
robinhood = prep_data('followers_count', 'friends_count', robinhood_followers, 'Robin Hood Foundation')
open_society = prep_data('followers_count', 'friends_count', open_society_followers, 'Open Society Foundation')
plot_scatter(robinhood, open_society)